In [31]:
from lxml import etree

In [32]:
xml=etree.parse('201312051000qf101.dat.xml.xml')

## 1 先获取根节点

xml中节点的目录结构如下：
- root:OceanObservatingDataFile  
    - BuoyageRpt  
        - BuoyInfo
            - Location
        - DateTime
        - HugeBuoyData
            - BuoyData

In [33]:
root=xml.getroot()

In [34]:
root

<Element OceanObservatingDataFile at 0x11206eac8>

In [5]:
for node in root:
    print(node)

<Element BuoyageRpt at 0x111fdff88>


In [6]:
list(root)

[<Element BuoyageRpt at 0x111fdff88>]

查看一下element的类型  
发现element是一个 ** lxml.etree._Element** 类型  
有几个问题：
- 如何获取element的name  
可以通过**.tag** 获取tag

In [7]:
type(root[0])

lxml.etree._Element

In [8]:
root[0].tag

'BuoyageRpt'

In [12]:
len(list(root))

1

使用* find *方法尝试一下

In [30]:
root.find('BuoyInfo')

### 重新梳理后的获取根节点的流程

- 对于读取的xml先获取root节点（获取到是一个element）
- 判断root节点的长度是否为1
- 对于是1的，在获取其tag的名称
- 判断是否为 * OceanObservatingDataFile *

In [40]:
root2=xml.getroot()

In [41]:
root2

<Element OceanObservatingDataFile at 0x11206eac8>

In [38]:
len(list(root2))==1

True

In [42]:
root2.tag

'OceanObservatingDataFile'

In [39]:
root2[0].tag=='BuoyageRpt'

True

In [43]:
root2[0].tag

'BuoyageRpt'

In [44]:
def getRoot():    
    root=xml.getroot()
    if len(list(root))==1:
        return (root if root.tag=='OceanObservatingDataFile' else None)

In [45]:
root=getRoot()

In [47]:
root

<Element OceanObservatingDataFile at 0x11206eac8>

## 2 从根节点之中找到buoyInfo子节点，读取*fubInfo*表中需要的数据

In [31]:
buoyageRpt=root.getchildren()

In [32]:
buoyageRpt

[<Element BuoyageRpt at 0x109470948>]

In [33]:
type(buoyageRpt)

list

In [34]:
type(buoyageRpt[0])

lxml.etree._Element

In [35]:
for node in buoyageRpt[0].getchildren():
    print(node.tag)

BuoyInfo
DateTime
HugeBuoyData


In [36]:
buoyageRpt[0]

<Element BuoyageRpt at 0x109470948>

In [37]:
buoyageRpt[0].find('BuoyInfo')

<Element BuoyInfo at 0x109475248>

### 2-1获取所有的属性值

In [38]:
fub_type=buoyageRpt[0].find('BuoyInfo').get('Type')
fub_code=buoyageRpt[0].find('BuoyInfo').get('id')
fub_name=buoyageRpt[0].find('BuoyInfo').get('Name')
fub_num=buoyageRpt[0].find('BuoyInfo').get('NO')
fub_kind=buoyageRpt[0].find('BuoyInfo').get('Kind')

'锚系浮标'

### 2-2 获取location，并通过正则匹配获取度和分

在**BuoyInfo**中还有一个子节点**Location**

In [161]:
location=buoyageRpt[0].find('BuoyInfo').getchildren()[0]

In [162]:
lon=location.get('longitude')
lat=location.get('latitude')

In [163]:
print(lat,lon)

38°19.98′N 120°9.85′E


此处使用正则表达式

In [164]:
import re

In [137]:
lat_point=re.match(r'^\d*',lat).group(0)
lat_point

'38'

取出分

注意**re.match**与**re.search**的区别

re.match只匹配字符串的开始，如果字符串开始不符合正则表达式，则匹配失败，函数返回None；  
re.search匹配整个字符串，直到找到一个匹配。

In [120]:
lat

'38°19.98′N'

In [75]:
# re.match(r'\d*)',lat)

In [127]:
lat

'38°19.98′N'

In [109]:
re.match(r'[0-9]+[.][0-9]{2}',lat)

In [128]:
re.match(r'[0-9]+[.]([0-9]{2}[′])',lat)

注意此处的问题，使用**re.match**无法找到匹配项，而**re.search**则可以

In [143]:
type(re.match(r'[0-9]+[.]([0-9]{2}\D\D)',lat))

NoneType

In [147]:
re.match(r'[0-9]+[.]([0-9]{2}\D\D)',lat)

In [156]:
re.match(r'\d+[°][0-9]+[.]([0-9]{2}\D\D)',lat)

<_sre.SRE_Match object; span=(0, 10), match='38°19.98′N'>

In [158]:
re.match(r'\d+[°][0-9]+[.]([0-9]{2})',lat).group(0)

'38°19.98'

In [160]:
lat.split('°')

['38', '19.98′N']

In [159]:
re.match(r'\d+[°][0-9]+[.]([0-9]{2})',lat).group(0).split('°')

['38', '19.98']

In [144]:
re.search(r'[0-9]+[.]([0-9]{2}\D\D)',lat)

<_sre.SRE_Match object; span=(3, 10), match='19.98′N'>

In [145]:
lat_min=re.search(r'[0-9]+[.]([0-9]{2})',lat).group(0)
lat_min

'19.98'

### 2-3 最终使用的获取经纬度的方法

In [167]:
lats=re.match(r'\d+[°][0-9]+[.]([0-9]{2})',lat).group(0).split('°')
lats

['38', '19.98']

In [168]:
lat=int(lats[0])+float(lats[1])/60

In [169]:
lat

38.333

In [170]:
lons=lats=re.match(r'\d+[°][0-9]+[.]([0-9]{2})',lon).group(0).split('°')
lons

['120', '9.85']

In [171]:
lon=int(lons[0])+float(lons[1])/60
lon

120.16416666666667

----------------------

## 3 从BuoyageRpt节点获取时间信息

xml中节点的目录结构如下：
- root:OceanObservatingDataFile  
    - BuoyageRpt  
        - BuoyInfo
            - Location
        - DateTime
        - HugeBuoyData
            - BuoyData

In [181]:
buoyageRpt[0]

<Element BuoyageRpt at 0x109470948>

In [182]:
buoyageRpt[0].find('DateTime')

<Element DateTime at 0x1094d8108>

In [183]:
buoyageRpt[0].find('DateTime').get('DT')

'201312051000'

--------------

## 4获取*BuoyData*节点信息

xml中节点的目录结构如下：
- root:OceanObservatingDataFile  
    - BuoyageRpt  
        - BuoyInfo
            - Location
        - DateTime
        - HugeBuoyData
            - BuoyData

In [184]:
buoyageRpt[0].find('HugeBuoyData')

<Element HugeBuoyData at 0x109475148>

In [186]:
BuoyData=buoyageRpt[0].find('HugeBuoyData').find('BuoyData')

In [187]:
BuoyData

<Element BuoyData at 0x1094ce888>

In [188]:
realtime_ws=BuoyData.get('WS')
realtime_wd=BuoyData.get('WD')
realtime_at=BuoyData.get('AT')
realtime_bp=BuoyData.get('BP')
realtime_hu=BuoyData.get('HU')
realtime_wt=BuoyData.get('WT')
realtime_sl=BuoyData.get('SL')
realtime_bg=BuoyData.get('BG')
realtime_ybg=BuoyData.get('YBG')
realtime_yzq=BuoyData.get('YZQ')

In [189]:
print(realtime_ws)
print(realtime_wd)
print(realtime_at)
print(realtime_bp)
print(realtime_hu)
print(realtime_wt)
print(realtime_sl)
print(realtime_bg)
print(realtime_ybg)
print(realtime_yzq)

12.5
34
5.9
1024.1
57
10.1
30.3
1.6
2.5
6.2
